In [2]:
from ris import pysqldb
from IPython.display import clear_output
from collections import defaultdict
from collections import defaultdict, namedtuple
import datetime 
import pandas as pd
import numpy as np
import os
from sqlalchemy import create_engine

clear_output()
timestamp = datetime.datetime.now().strftime('%Y-%m-%d_%H:%M')
ts = datetime.datetime.now().strftime('%Y-%m-%d')

print 'Notebook run: {}'.format(timestamp)
print os.getcwd()

Notebook run: 2020-09-24_14:20
E:\RIS\Staff Folders\Samuel\Requests\SH\Safety_Data_Viewer


In [3]:
#Database connection
cdb = pysqldb.DbConnect(server='DOTDEVRHPGSQL01', database='CRASHDATA', type='PG')



Additional database connection details required:
User name (crashdata):soge
Password (crashdata)········
Database connection (PG) to CRASHDATA on DOTDEVRHPGSQL01 - user: soge 
Connection established 2020-09-24 14:23:23


In [4]:
cdb.connect()

Database connection (PG) to CRASHDATA on DOTDEVRHPGSQL01 - user: soge 
Connection established 2020-09-24 14:23:23


### Ped Injury Summary

In [11]:
ped_inj_sum = cdb.dfquery("""

WITH data aS(
SELECT * FROM nysdot_all 
WHERE nodeid::int = 18976
and case_yr between 2014 and 2018
and accd_typ = '02' 
and num_of_inj > 0
)

,data2 AS 
(SELECT sum(num_of_inj)
FROM data)

SELECT *
FROM data

""")

In [12]:
ped_inj_sum

,gid,case_num,case_yr,ref_mrkr,accd_dte,road_sys,num_of_fat,num_of_inj,reportable,police_dep,...,alis_node,alis_mastr,x,y,st_x,st_y,wrong_pct,cleanname,manual,revision_notes
0,NaN,35893402,2015,None,2015-06-22,12,0,1,Y,3030,...,9999,9999,997183.368808949,187255.354857776,997183.310417235,187255.5512802,None,None,False,None
1,377718.0,35531813,2014,None,2014-10-04,12,0,1,Y,03030,...,9999,9999,997183.368806362,187255.354869857,997183.310417235,187255.5512802,None,BEDFORD AVENUE,False,None
2,353192.0,35335952,2014,None,2014-05-09,12,0,1,Y,03030,...,9999,9999,997183.368806362,187255.354869857,997183.310417235,187255.5512802,None,BEDFORD AVENUE,False,None
3,353189.0,35712846,2014,None,2014-12-26,12,0,1,Y,03030,...,9999,9999,997183.368806362,187255.354869857,997183.310417235,187255.5512802,None,None,False,None
4,NaN,36784868,2017,None,2017-06-27,12,0,1,Y,3030,...,9999,9999,997183.305945238,187255.772390584,997183.310417235,187255.5512802,None,BEDFORD AVENUE,False,None
5,NaN,36741634,2017,None,2017-05-25,12,0,1,Y,3030,...,9999,9999,997183.305945238,187255.772390584,997183.310417235,187255.5512802,None,BEDFORD AVENUE,False,None


In [7]:
bike_inj_sum = cdb.dfquery("""

WITH data aS(
SELECT * FROM nysdot_all 
WHERE nodeid::int = 18976
and case_yr between 2014 and 2018
and accd_typ = '03'
and num_of_inj > 0
)

,data2 AS
(SELECT sum(num_of_inj)
FROM data)

""")

In [8]:
bike_inj_sum

,sum
0,3


In [9]:
mvo_inj_sum = cdb.dfquery("""

WITH data AS(
SELECT * FROM nysdot_all 
WHERE nodeid::int = 18976
and case_yr between 2014 and 2018
and accd_typ != '02' 
and accd_typ != '03'
and num_of_inj > 0
)

,data2 AS
(SELECT sum(num_of_inj)
FROM data)

SELECT * 
FROM data

""")

In [10]:
mvo_inj_sum

,sum
0,30


In [ ]:
veh_type_inj_sum = cdb.dfquery("""
WITH data AS(
SELECT nys.*, nys_v.veh_seq_num, nys_v.veh_typ
FROM nysdot_all nys
JOIN nysdot_vehicle  nys_v
ON nys.crashid = nys_v.crashid
WHERE nodeid::int = 18976
and nys.case_yr between 2014 and 2018
)


SELECT DISTINCT crashid, case_num, veh_typ 
FROM data
WHERE veh_typ =  '2'
AND num_of_inj != 0
""")